In [163]:
import pandas as pd
import numpy as np
import re
import dateutil.parser as dparser
from tqdm import tqdm
from tqdm.auto import tqdm
import tiktoken
import openai
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
OPENAI_KEY = os.getenv('OPENAI_KEY')
openai.api_key = OPENAI_KEY

In [7]:
def find_max_len(df):
    max_len = 0
    for text in df['text']:
        if len(text) > max_len:
            max_len = len(text)
    return max_len

## Dataframes

In [153]:
df_nine_for_news = pd.read_csv('nine_for_news_clean.csv').drop(['index'], axis=1).dropna()
df_nine_for_news['source'] = 'Nine for News'

# Filter all non-article entries
df_nine_for_news = df_nine_for_news[df_nine_for_news['url'].str.contains("2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021|2022|2023|tag|categorie|page")==False]


df_nine_for_news

,text,date,url,source
0,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/daar-gaan-we-weer-w...,Nine for News
1,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/kijk-gevaccineerde-...,Nine for News
2,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/advocaat-start-same...,Nine for News
3,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/aivd-gebruikt-nepfo...,Nine for News
4,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,26-05-2023,https://www.ninefornews.nl/oud-vvder-doet-boek...,Nine for News
...,...,...,...,...
643,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,20-07-2016,https://www.ninefornews.nl/chemtrails-een-goed...,Nine for News
644,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,8-07-2016,https://www.ninefornews.nl/politicus-regering-...,Nine for News
645,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,1-07-2016,https://www.ninefornews.nl/chemtrails-vliegtui...,Nine for News
646,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,26-05-2016,https://www.ninefornews.nl/expert-deze-beelden...,Nine for News


In [154]:
df_indymedia= pd.read_csv('indymedia_clean.csv').drop(['Unnamed: 0', 'index'], axis=1).dropna()
df_indymedia['source'] = 'Indy media'

# Filter all non-article entries
df_indymedia = df_indymedia[df_indymedia['url'].str.contains("node")==True]
df_indymedia

,text,date,url,source
0,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,19-5-2023,https://www.indymedia.nl/node/53410,Indy media
1,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,17-5-2023,https://www.indymedia.nl/node/53399,Indy media
2,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,13-5-2023,https://www.indymedia.nl/node/53386,Indy media
3,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,5-12-2023,https://www.indymedia.nl/node/53377,Indy media
4,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,5-5-2023,https://www.indymedia.nl/node/53361,Indy media
...,...,...,...,...
77,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,5-4-2023,https://www.indymedia.nl/node/53356,Indy media
78,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,20-3-2023,https://www.indymedia.nl/node/53182,Indy media
79,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,19-5-2023,https://www.indymedia.nl/node/53414,Indy media
80,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,5-10-2023,https://www.indymedia.nl/node/53371,Indy media


In [155]:
df_niburu = pd.read_csv('niburu_clean.csv').drop(['Unnamed: 0', 'index'], axis=1).dropna()
df_niburu['source'] = 'Niburu'

# Filter all non-article entries
df_niburu = df_niburu[df_niburu['url'].str.contains("start")==False]
df_niburu

,text,date,url,source
0,Toggle navigationInfoHet heeft er alle schijn ...,27-5-2023,https://niburu.co/,Niburu
1,Toggle navigationInfoHet heeft er alle schijn ...,27-5-2023,https://niburu.co/binnenland/18595-er-lijkt-ie...,Niburu
2,Toggle navigationInfoVoordat zo goed als alle ...,27-5-2023,https://niburu.co/het-complot/18594-complotden...,Niburu
3,Toggle navigationInfo Sommigen zullen het toev...,27-5-2023,https://niburu.co/het-complot/18593-nieuwe-asi...,Niburu
4,Toggle navigationInfo De AIVD waarschuwt voor ...,26-5-2023,https://niburu.co/het-complot/18591-gaat-overh...,Niburu
...,...,...,...,...
93,Toggle navigationInfo Misschien wordt het eind...,23-4-2023,https://niburu.co/het-complot/18493-red-de-pla...,Niburu
94,Ministers liegen alsof het gedrukt staat en de...,23-5-2023,https://niburu.co/binnenland/18583-steeds-meer...,Niburu
95,Toggle navigationInfoDe nep coronapandemie lig...,14-5-2023,https://niburu.co/het-complot/18555-tijdlijn-n...,Niburu
96,"Toggle navigationInfoDit is het niet, omdat he...",19-7-2021,https://niburu.co/binnenland/16500-het-parleme...,Niburu


### Privacynews doesn't really have any extreme content as it is only concerned with news about privacy

In [111]:
# df_privacynieuws = pd.read_csv('privacynieuws_clean.csv').drop(['Unnamed: 0', 'index'], axis=1).dropna()
# df_privacynieuws['source'] = 'Privacy nieuws'
# find_max_len(df_privacynieuws)

35020

In [157]:
df_frontnieuws = pd.read_csv('frontnieuws_clean.csv').drop(['Unnamed: 0', 'index'], axis=1).dropna()
df_frontnieuws['source'] = 'Frontnieuws'

# Filter all non-article entries
df_frontnieuws= df_frontnieuws[df_frontnieuws['url'].str.contains("category|#comments")==False]
df_frontnieuws

,text,date,url,source
1,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\nCOVID-1...,27-5-2023,https://www.frontnieuws.com/covid-19-cartoons/,Frontnieuws
7,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n - - A...,27-5-2023,https://www.frontnieuws.com/,Frontnieuws
8,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n5DEen v...,27-5-2023,https://www.frontnieuws.com/hoe-het-westen-ver...,Frontnieuws
9,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\nFrontni...,27-5-2023,https://www.frontnieuws.com/author/frontnieuws...,Frontnieuws
10,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n1237DHe...,27-5-2023,https://www.frontnieuws.com/russische-tanks-be...,Frontnieuws
...,...,...,...,...
491,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n3300UEe...,29-4-2021,https://www.frontnieuws.com/65-studies-tonen-a...,Frontnieuws
493,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n9229BDe...,15-9-2022,https://www.frontnieuws.com/kijk-eu-parlements...,Frontnieuws
495,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n8741EHe...,12-10-2021,https://www.frontnieuws.com/gevaccineerde-pilo...,Frontnieuws
497,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n6390DIk...,23-8-2022,https://www.frontnieuws.com/in-het-hoofd-van-e...,Frontnieuws


# Skipping these for now as there is no date info available
The dataset is also quite large already. We don't want the GPT costs to be too high.

In [110]:
df_de_andere_krant = pd.read_csv('results_de_andere_krant.csv').dropna()
df_de_andere_krant
find_max_len(df_de_andere_krant)

2678

In [109]:
df_transitieweb = pd.read_csv('results_transitieweb.csv').dropna()
df_transitieweb
find_max_len(df_transitieweb)

35676

# Full dataset

In [158]:
frames = [df_nine_for_news, df_indymedia, df_niburu, df_frontnieuws]
final_df = pd.concat(frames)
final_df

,text,date,url,source
0,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/daar-gaan-we-weer-w...,Nine for News
1,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/kijk-gevaccineerde-...,Nine for News
2,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/advocaat-start-same...,Nine for News
3,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/aivd-gebruikt-nepfo...,Nine for News
4,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,26-05-2023,https://www.ninefornews.nl/oud-vvder-doet-boek...,Nine for News
...,...,...,...,...
491,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n3300UEe...,29-4-2021,https://www.frontnieuws.com/65-studies-tonen-a...,Frontnieuws
493,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n9229BDe...,15-9-2022,https://www.frontnieuws.com/kijk-eu-parlements...,Frontnieuws
495,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n8741EHe...,12-10-2021,https://www.frontnieuws.com/gevaccineerde-pilo...,Frontnieuws
497,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n6390DIk...,23-8-2022,https://www.frontnieuws.com/in-het-hoofd-van-e...,Frontnieuws


# GPT
## Tokenizing

In [159]:
def tokenize_text(df):
    for i, row in enumerate(df):
        text = df['text'][i]
        tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')
        encoded_text = tokenizer.encode(text)

        # Max amount of tokens for a request is a little more than 4000
        # this includes both the text and the instructions given to GPT
        # Lets remove the texts > 3000 tokens
        if len(encoded_text) > 3000:
            encoded_text = pd.NA

    return encoded_text

In [185]:
def tokenize_text(df):
    for i, row in enumerate(df['text']):
        text = str(row)
        tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')
        encoded_text = tokenizer.encode(text)

        # Max amount of tokens for a request is a little more than 4000
        # this includes both the text and the instructions given to GPT
        # Lets remove the texts > 3000 tokens
        if len(encoded_text) > 3000:
            encoded_text = pd.NA

        df['encoded text'][i] = encoded_text

    return df

In [192]:
test = final_df[0:10]
test['encoded text'] = ''
tokenize_text(test)

C:\Users\ceesm\AppData\Local\Temp\ipykernel_5308\4214078122.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['encoded text'] = ''
C:\Users\ceesm\AppData\Local\Temp\ipykernel_5308\3694886075.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['encoded text'][i] = encoded_text
C:\Users\ceesm\AppData\Local\Temp\ipykernel_5308\3694886075.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

,text,date,url,source,tokenized_text,encoded text
0,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/daar-gaan-we-weer-w...,Nine for News,,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
1,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/kijk-gevaccineerde-...,Nine for News,,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
2,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/advocaat-start-same...,Nine for News,,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
3,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/aivd-gebruikt-nepfo...,Nine for News,,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
4,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,26-05-2023,https://www.ninefornews.nl/oud-vvder-doet-boek...,Nine for News,,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
5,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,26-05-2023,https://www.ninefornews.nl/femke-halsema-breng...,Nine for News,,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
6,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,26-05-2023,https://www.ninefornews.nl/omt-lid-beweert-dat...,Nine for News,,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
7,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,26-05-2023,https://www.ninefornews.nl/kijk-klaus-schwab-d...,Nine for News,,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
8,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,26-05-2023,https://www.ninefornews.nl/dit-is-de-belangrij...,Nine for News,,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
9,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,26-05-2023,https://www.ninefornews.nl/ergernis-om-krokodi...,Nine for News,,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."


1261